## 1. 이미지 파일 예측

In [1]:
import numpy as np

In [3]:
img=np.array([[2,0,2,1,3,2],
              [0,2,0,2,2,2],
              [1,0,1,3,1,1],
              [0,0,1,1,1,0],
              [0,1,3,4,1,0],
              [0,1,0,0,5,2]])
img

array([[2, 0, 2, 1, 3, 2],
       [0, 2, 0, 2, 2, 2],
       [1, 0, 1, 3, 1, 1],
       [0, 0, 1, 1, 1, 0],
       [0, 1, 3, 4, 1, 0],
       [0, 1, 0, 0, 5, 2]])

In [7]:
#이미지 파일에 제로 패딩
zero_p = np.array([[0,0,0,0,0,0,0,0],
                   [0,2,0,2,1,3,2,0],
                   [0,0,2,0,2,2,2,0],
                   [0,1,0,1,3,1,1,0],
                   [0,0,0,1,1,1,0,0],
                   [0,0,1,3,4,1,0,0],
                   [0,0,1,0,0,5,2,0],
                   [0,0,0,0,0,0,0,0]])
#filter= np.array([[1,0,0],[0,1,0],[0,0,1]])
zero_p

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
#필터 거침
filtered = np.array([[4,0,4,3,5,2],
                    [0,5,3,5,4,5],
                    [1,1,4,4,3,3],
                    [1,4,5,3,4,1],
                    [1,1,3,10,4,1],
                    [0,1,1,3,9,3]])
filtered

array([[ 4,  0,  4,  3,  5,  2],
       [ 0,  5,  3,  5,  4,  5],
       [ 1,  1,  4,  4,  3,  3],
       [ 1,  4,  5,  3,  4,  1],
       [ 1,  1,  3, 10,  4,  1],
       [ 0,  1,  1,  3,  9,  3]])

In [9]:
#맥스풀링함
max_p = np.array([[5,5,5],
                  [4,5,4],
                  [1,10,9]])
max_p                  

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

## 2. 코드 활용 문제

### 2.1) layer kernel size, stride 변화

원래 모델 먼저 확인해보자

In [5]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [8]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [9]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,
                                          shuffle=True, drop_last=True)

In [10]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1,torch.nn.ReLU(),torch.nn.Dropout(p=1 - self.keep_prob))

        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [11]:
# instantiate CNN model
model = CNN().to(device)

In [12]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.190713644
[Epoch:    2] cost = 0.0520608276
[Epoch:    3] cost = 0.0376487821
[Epoch:    4] cost = 0.0285026077
[Epoch:    5] cost = 0.0243023224
[Epoch:    6] cost = 0.0195404254
[Epoch:    7] cost = 0.0160752013
[Epoch:    8] cost = 0.0142789194
[Epoch:    9] cost = 0.0117072025
[Epoch:   10] cost = 0.0123375095
[Epoch:   11] cost = 0.0106206443
[Epoch:   12] cost = 0.0107046869
[Epoch:   13] cost = 0.00735590933
[Epoch:   14] cost = 0.00858276151
[Epoch:   15] cost = 0.00802615285
Learning Finished!


In [14]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\dbdms\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\dbdms\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9864000082015991


layer1의 kernel size를 4로, layer 3의 stride를 2로 늘리자. 

$$O_h=floor((Ih−Kh+2P)/S+1)$$
$$O_w=floor((Iw−Kw+2P)/S+1)$$
floor는 소수점 버리는 함수

In [20]:
class CNN2(torch.nn.Module):

    def __init__(self):
        super(CNN2, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 27, 27, 32)
        #    Pool     -> (?, 13, 13, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 13, 13, 32)
        #    Conv      ->(?, 13, 13, 64)
        #    Pool      ->(?, 6, 6, 64) 
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 6, 6, 64)
        #    Conv      ->(?, 3, 3, 128)
        #    Pool      ->(?, 2, 2, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear(2 * 2 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1,torch.nn.ReLU(),torch.nn.Dropout(p=1 - self.keep_prob))

        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

### 2.2) Layer 3 output size?: 2x2

### 2.3) Cost와 Accuracy가 어떻게 변할까? 
: 늘지 않을까?

### 2.4) 코딩하여 확인해 보라!

In [21]:
model= CNN2().to(device)

In [22]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [23]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.248539209
[Epoch:    2] cost = 0.0596137792
[Epoch:    3] cost = 0.0410695225
[Epoch:    4] cost = 0.0334572382
[Epoch:    5] cost = 0.0267660208
[Epoch:    6] cost = 0.0225592311
[Epoch:    7] cost = 0.020421857
[Epoch:    8] cost = 0.0166569408
[Epoch:    9] cost = 0.0157424528
[Epoch:   10] cost = 0.0147862835
[Epoch:   11] cost = 0.010000376
[Epoch:   12] cost = 0.0100708473
[Epoch:   13] cost = 0.00974999741
[Epoch:   14] cost = 0.00954514928
[Epoch:   15] cost = 0.00825454015
Learning Finished


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9807000160217285


cost 오르고 accuracy 조금 감소